<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/hungarian/python/dxf_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DXF fájlok kezelése Pythonból

A DXF fájlok az AutoCAD program szöveges adatcsere formátuma.
Tartalma megegyezik a DWG fájlok tartalmával, de azokkal ellentétben a tartalma mindenki számára elérhetően dokumentált(https://docs.fileformat.com/cad/dxf/). Így tartalmuk
egy programmal is értelmezhetők illetve létrehozható.

A DXF fájlok tartalma fejezetekre bomlik (HEADER, TABLES, ENTITIES, stb.). Az egyes adatokat két sorban tartalmazza, az első sorban egy kód, a másodikban pedig egy szöveges vagy numerikus érték következik.
A kód határozza meg, hogyan kell értelmezni a következő adatot (pl. 10 - x koordináta, 2 - név blokknév/rétegnév a kontextustól függően).
A következő kis részletben (ENTITIES blokk) a kódok és értékek után megjegyzéseket írtunk, ezek nem részei a DXF fájlnak.

```
0                            ; új entitás kezdete
LINE                         ; entitás típusa
5                            ; entitás azonosító
8F4                          ; hexadecimális azonosító érték
100
AcDbEntity
8                            ; réteg
GEOD_KERITES_KAPU            ; réteg neve
6                            ; vonaltípus
Continuous                   ; folytonos
62                           ; színkód
56
100
AcDbLine
39
0.
10                           ; kezdőpont x koordináta
590408.86
20                           ; kezdőpont y koordináta
167862.39
30                           ; kezdőpont z koordináta
0.
11                           ; végpont x koordináta
590409.82
21                           ; végpont y koordináta
167861.88
31                           ; végpont z koordináta
0.
```

## DXF file létrehozása koordinátalistából

Egy CSV fájlban adott pontszámok és koordináták alapján hozzunk létre DXF fájlt.

Először letöltünk egy CSV fájlt a koordinátákkal és írjuk ki az első néhány sorát.

In [2]:
!wget -q https://raw.githubusercontent.com/OSGeoLabBp/tutorials/master/english/data_processing/lessons/code/coo_list.csv
!head coo_list.csv

﻿base_1,667568.278,271801.865,252.176
1,655738.628,257962.892,117.020
600,655738.779,257962.765,117.019
601,655739.312,257962.986,117.049
602,655921.110,257840.450,117.644
603,655922.395,257844.152,117.472
604,655925.717,257853.584,117.733
605,655928.146,257860.510,117.552
606,655930.936,257868.434,117.240
607,655933.516,257874.923,116.623


Olvassuk be a CSV fájl tartalmát egy *pandas* adatkeretbe! Mint az előző listából látható, nincs fejléc sor a fájlban.

In [10]:
import pandas as pd
names = ['psz', 'y', 'x', 'z']
fname = 'coo_list.csv'
try:
    df = pd.read_csv(fname, names=names)
except FileNotFoundError:
    print(f'Nincs ilyen fájl:{fname}')

In [11]:
df.head()

,psz,y,x,z
0,base_1,667568.278,271801.865,252.176
1,1,655738.628,257962.892,117.020
2,600,655738.779,257962.765,117.019
3,601,655739.312,257962.986,117.049
4,602,655921.110,257840.450,117.644


Hozzuk létre a DXF fájlt és írjuk ki a minimálisan szükséges kezdő információkat.

In [14]:
try:
    f_dxf = open('coo_list.dxf', 'w')
    print('  0\nSECTION\n  2\nENTITIES', file=f_dxf)
except:
    print('Nem sikerült a fájlba írni')

Menjünk végig egyesével az adatkeret sorain és írjunk ki egy pont entitást és egy szöveg entitást (pontszám) a DXF fájlba.

In [15]:
layer_p = "points"
layer_t = "point_id"
for index, row in df.iterrows():
    print(f"  0\nPOINT\  8\n{layer_p}\n 10\n{row['x']:.3f}\n 20\n{row['y']:.3f}\n 30\n{row['z']:.3f}", file=f_dxf)
    print(f"  0\nTEXT\  8\n{layer_t}\n 10\n{row['x']+0.3:.3f}\n 20\n{row['y']+0.3:.3f}\n 30\n{row['z']:.3f}\n  1\n{row['psz']}\n", file=f_dxf)

Végül zárjuk le a DXF fájlt.

In [17]:
print("  0\nENDSEC\n  0\nEOF", file=f_dxf)
f_dxf.close()